In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, LeaveOneOut, StratifiedKFold
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('glass.csv')

In [4]:
df.Type.value_counts()

Type
2    76
1    70
7    29
3    17
5    13
6     9
Name: count, dtype: int64

In [6]:
df['Type']=df['Type'].apply(str) # 정답데이터 str로 형변환

In [8]:
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [9]:
df.columns

Index(['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type'], dtype='object')

In [10]:
#train/ test로 나누기 위해서는 독립변수 종속변수 나눠야 한다. (정답데이터를 지정해야 합니다.)
df_x = df[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']] #독립변수
df_y = df['Type'] #종속변수 y값 label 

In [11]:
df_x

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0
...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0


In [12]:
df_y

0      1
1      1
2      1
3      1
4      1
      ..
209    7
210    7
211    7
212    7
213    7
Name: Type, Length: 214, dtype: object

In [13]:
#랜덤포레스트만 가지고 오기!
rfc =RandomForestClassifier(n_estimators=200)

In [15]:
##train, test로 나누기!
# train_test_split(x값, y값, 다른 하이퍼파라미터 넣기 test_size, random_state = )
#train/ test 2개로 나눠진다!
#train_x train_y / test_x test_y값이 있다.
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, random_state=111)

## 기본_train_Test_split 사용_ 기본 데이터를 나눈 상태로 모델에 학습시켜보기!

In [20]:
rfc_model=rfc.fit(x_train, y_train) ## 모델에 트레인 데이터만 넣어서 학습

In [22]:
pred_y=rfc_model.predict(x_test) #예측을 하는 것! ( train 가지고 학습해서 y 값을 예측하는 것)

In [23]:
pred_y

array(['1', '5', '7', '2', '7', '1', '7', '1', '2', '1', '5', '1', '5',
       '1', '3', '1', '1', '2', '1', '2', '1', '1', '7', '2', '2', '2',
       '2', '2', '1', '5', '1', '1', '1', '2', '1', '2', '7', '2', '1',
       '1', '5', '3', '3', '1', '5', '2', '1', '2', '7', '1', '1', '2',
       '2', '1', '2', '1', '1', '1', '1', '7', '1', '2', '2', '2', '3'],
      dtype=object)

In [24]:
y_test

65     1
175    5
192    7
92     2
195    7
      ..
38     1
183    6
142    2
83     2
149    3
Name: Type, Length: 65, dtype: object

In [25]:
#정확도
base_ac=accuracy_score(y_test, pred_y)

In [26]:
base_ac # 80% 수준의 값이 나왔다!

0.8

## k-fold로 진행해 보기!  cross_val_score 함수를 사용! 
- cross_val_score

In [27]:
#랜덤포레스트만 가지고 오기!
rfc =RandomForestClassifier(n_estimators=200)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, random_state=111)

#검증을 진행해야 한다 cross_val_score 함수로 k-fold 진행하는 것!
#cross_val_score(사용할 모델, train_x, train_y, cv= 폴드를 몇 개 만들 것인가?)
kfold_score_1=cross_val_score(rfc, x_train,y_train, cv = 5)

In [28]:
kfold_score_1

array([0.76666667, 0.76666667, 0.86666667, 0.76666667, 0.75862069])

# 필수과제 1
- 타이타닉데이터를 가지고 직접 basic train_test_split을 사용, k-fold로 사용 두 개의 정확도를 비교해 주세요!
- 피처 엔지니어링을 추가해서 본인의 생각하는 1개의 변수는 추가해보고 실제 성능이 어떤지 확인해 보자!
- 기존의 타이타닉데이터에서 -> 파생변수 최소 1개는 꼭 만들기!

# 필수과제 2
- 1. base 컬럼만 사용한 타이타닉데이터 base train_Test_split 정확도, k-fold 정확도
- 2. 내가 만든 파생변수 컬럼 추가한 타이타닉데이터 base train_Test_split 정확도, k-fold 정확도
- 우리의 목표는 base 보다는 높은 정확도를 만들어보자!